<a href="https://colab.research.google.com/github/Nikshitha987/Historical-Images/blob/main/Historical_Images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FiLM(nn.Module):
    def __init__(self, feat_dim, emb_dim):
        super().__init__()
        self.fc = nn.Linear(emb_dim, feat_dim*2)  # gamma and beta

    def forward(self, x, emb):
        # x: (B, C, H, W), emb: (B, emb_dim)
        params = self.fc(emb)  # (B, 2*C)
        gamma, beta = params.chunk(2, dim=1)
        gamma = gamma.unsqueeze(-1).unsqueeze(-1)
        beta = beta.unsqueeze(-1).unsqueeze(-1)
        return x * (1 + gamma) + beta

class BasicBlock(nn.Module):
    def __init__(self, in_ch, out_ch, emb_dim=None):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True)
        )
        self.film = FiLM(out_ch, emb_dim) if emb_dim is not None else None

    def forward(self, x, emb=None):
        x = self.conv(x)
        if self.film is not None and emb is not None:
            x = self.film(x, emb)
        return x

class CondUNet(nn.Module):
    def __init__(self, emb_dim=64):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings=10, embedding_dim=emb_dim)  # adjust num periods
        self.down1 = BasicBlock(1, 64, emb_dim)
        self.down2 = BasicBlock(64, 128, emb_dim)
        self.down3 = BasicBlock(128, 256, emb_dim)
        self.pool = nn.MaxPool2d(2)
        self.up2 = BasicBlock(256+128, 128, emb_dim)
        self.up1 = BasicBlock(128+64, 64, emb_dim)
        self.out_conv = nn.Conv2d(64, 2, 1)  # predict AB channels (Lab)

    def forward(self, L, period_ids):
        # L: (B,1,H,W) grayscale (L channel normalized), period_ids: (B,)
        emb = self.emb(period_ids)  # (B, emb_dim)

        d1 = self.down1(L, emb)
        p1 = self.pool(d1)
        d2 = self.down2(p1, emb)
        p2 = self.pool(d2)
        d3 = self.down3(p2, emb)

        u2 = F.interpolate(d3, scale_factor=2, mode='bilinear', align_corners=False)
        u2 = torch.cat([u2, d2], dim=1)
        u2 = self.up2(u2, emb)

        u1 = F.interpolate(u2, scale_factor=2, mode='bilinear', align_corners=False)
        u1 = torch.cat([u1, d1], dim=1)
        u1 = self.up1(u1, emb)

        ab = self.out_conv(u1)  # (B,2,H,W)
        return ab


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving 1382906.png to 1382906.png


In [ ]:
from PIL import Image
import numpy as np
import torch
import io

# Replace 'your_image.jpg' with the actual filename you uploaded
filename = list(uploaded.keys())[0]
print("Using image:", filename)

# Load the image
img = Image.open(io.BytesIO(uploaded[filename])).convert("L")

# Resize to model size
img = img.resize((256, 256))

# Convert to tensor (L channel)
L = np.array(img) / 255.0
L = torch.tensor(L).float().unsqueeze(0).unsqueeze(0)  # shape: (1,1,256,256)


Using image: 1382906.png


In [ ]:
# Example: 0 = 1920s, 1 = 1930s, 2 = WWII, etc.
period_id = torch.tensor([2])   # WWII example


In [ ]:
model = CondUNet()       # Replace with your trained model
model.eval()

with torch.no_grad():
    pred_ab = model(L, period_id)   # output AB channels


In [ ]:
import cv2
import numpy as np

# Convert L + AB → RGB
L_np = L.squeeze().numpy() * 100   # L in 0-100
AB_np = pred_ab.squeeze().numpy() * 128  # scale back from [-1,1] if used

lab = np.zeros((256, 256, 3), dtype=np.float32)
lab[:,:,0] = L_np
lab[:,:,1:] = AB_np.transpose(1,2,0)

rgb = cv2.cvtColor(lab.astype(np.uint8), cv2.COLOR_LAB2RGB)

# Save output
cv2.imwrite("colorized_output.png", cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR))
print("Saved: colorized_output.png")


Saved: colorized_output.png


In [ ]:
from google.colab import files
files.download("colorized_output.png")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
filename = list(uploaded.keys())[0]
img = Image.open(io.BytesIO(uploaded[filename])).convert("L")


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving 1382906.png to 1382906 (1).png


In [ ]:
from PIL import Image
import numpy as np
import torch
import io

filename = list(uploaded.keys())[0]
print("Using image:", filename)

# Load image as Grayscale (L channel)
img = Image.open(io.BytesIO(uploaded[filename])).convert("L")

# Resize to model size
img = img.resize((256, 256))

# Convert to tensor
L = np.array(img) / 255.0
L = torch.tensor(L).float().unsqueeze(0).unsqueeze(0)   # (1,1,256,256)


Using image: 1382906 (1).png


In [ ]:
# Example: 0=1920s, 1=1930s, 2=WWII, etc.
period_id = torch.tensor([2])  # choose the period you want


In [ ]:
model = CondUNet()            # replace with your trained model checkpoint
model.eval()

with torch.no_grad():
    pred_ab = model(L, period_id)     # Generate AB (color) channels


In [ ]:
import cv2

# Convert back to LAB image
L_img = (L.squeeze().numpy() * 100).astype("uint8")          # L in [0,100]
AB_img = (pred_ab.squeeze().numpy() * 128).astype("int16")   # A/B in [-128,128]

lab = np.zeros((256, 256, 3), dtype=np.int16)
lab[:,:,0] = L_img
lab[:,:,1] = AB_img[0]
lab[:,:,2] = AB_img[1]

# Convert LAB → RGB
rgb = cv2.cvtColor(lab.astype("uint8"), cv2.COLOR_LAB2RGB)

# Save output
cv2.imwrite("colorized_output.png", cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR))
print("Saved color image as colorized_output.png")


Saved color image as colorized_output.png


In [ ]:
from google.colab import files
files.download("colorized_output.png")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>